Let $\tau_i$ denote the membrane time constant of neuron $i$. Then, up to the approximations related to the assumption of being in an Asynchronous Irregular state, we have the following forumalas for the mean and SD (standard deviation) of the input to different neurons. 

$\mu_i = \tau_i \sum_j W_{ij} r_j + h_i$

$\sigma^2_i = \tau_i \sum_j W_{ij}^2 r_j + \xi^2_i$

where $h_i$ and $\xi_i$ are the mean (which is positive) and SD of the external input, respectively.

And then the output firing rate of each neuron is given by 

$r_i = \Phi(\mu_i, \sigma_i) \qquad\qquad\qquad$     (1)

If I plug in the first two equations (where each is really $N$ equations) in the bottom one (again, really $N$ equations), then I obtain a system of $N$ equations for the $N$ unknowns, namely the $r_i$.  (part of) Your tasks is to solve this ssytem of equations. To do that, we will first promote these to time-dependent ODE's. These are auxiliary tools for us to find the fixed point, and are not really physical:

$T_i\frac{dr_i}{dt} + r_i = \Phi(\mu_i, \sigma_i)$

We will then solve these numerically using the Euler method (see my `utils_for_max.py`), and hope that the ODE's will converge onto a fixed point, which satisfies (1). The Euler discretization leads to

$r_i(t+1) = (1 - \alpha_i) r_i(t) + \alpha_i \Phi(\mu_i, \sigma_i)$

where I defined $\alpha_i = \frac{\delta t}{T_i}$. But to use `utils_for_max.Euler2fixedpt` you don't really need to do this, but you need to give it (as `dxdt`) the following:

$T_i^{-1} (-r_i + \Phi(\mu_i, \sigma_i))$ with the above expressions for $\mu$ and $\sigma$ plugged in. 


In the rest, I assume that you have already substituted the expressions for $\mu_i$ and $\sigma_i$, but won't write them explicitly for brevity.

In the 



In [1]:
import numpy as np
from model import Model, theta_diff, block_matrix
from utils_for_max import Phi, Euler2fixedpt
import utils_for_max

In [2]:
# Define model with N_E and N_I
model = Model(16, 8)

# Assigning parameters J, P, w^2
zeros = np.zeros((2, 2))
model.set_parameters(zeros, zeros, zeros)

print(model.set_inputs(1, 0))

(array([2.00000000e+01, 7.65222376e+00, 4.44728586e-01, 4.37964718e-03,
       8.72839838e-06, 4.47054248e-09, 7.87355835e-13, 6.64719496e-17,
       3.85749970e-21, 2.23858394e-25, 1.88990838e-29, 3.32852310e-33,
       1.70481493e-36, 3.39760335e-39, 3.34592926e-41, 1.94457250e-42,
       2.00000000e+01, 4.44728586e-01, 8.72839838e-06, 7.87355835e-13,
       3.85749970e-21, 1.88990838e-29, 1.70481493e-36, 3.34592926e-41]), array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5.]))


In [3]:
model.generate_network()
model.solve_fixed_point()  # Is producing overflow error!
model.r

array([3.97574827e+01, 3.21439131e-01, 7.37083332e-05, 3.84279652e-05,
       3.79231689e-05, 3.81767159e-05, 1.63847055e-05, 1.62703777e-05,
       1.63846522e-05, 2.47141342e-05, 5.68648525e-05, 5.68648373e-05,
       1.63847550e-05, 5.72429358e-05, 3.79226381e-05, 3.81767190e-05,
       4.42546945e+01, 7.37084005e-05, 2.47144344e-05, 1.63846977e-05,
       1.63847191e-05, 1.62704212e-05, 3.81768552e-05, 3.81768268e-05])

In [4]:
model.r_change()

array([3.97574827e+01, 3.21439132e-01, 7.37083339e-05, 3.84279656e-05,
       3.79231692e-05, 3.81767163e-05, 1.63847055e-05, 1.62703777e-05,
       1.63846522e-05, 2.47141342e-05, 5.68648531e-05, 5.68648378e-05,
       1.63847550e-05, 5.72429364e-05, 3.79226385e-05, 3.81767194e-05,
       4.42546945e+01, 7.37084005e-05, 2.47144344e-05, 1.63846977e-05,
       1.63847191e-05, 1.62704212e-05, 3.81768553e-05, 3.81768268e-05])

In [5]:
curves = model.get_tuning_curves()